In [7]:
"""
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
"""
#vérifier d'avoir installer pyserial

import time
import numpy as np
from Thymio import Thymio

#Finding the right port
#port=Thymio.serial_default_port()
#print(port)
# By Cable
th = Thymio.serial(port="COM8", refreshing_rate=0.1)
# By Wireless
#th = Thymio.serial(port="\\.\COM5", refreshing_rate=0.1)
#dir(th)

SerialException: could not open port 'COM8': FileNotFoundError(2, 'Le fichier spécifié est introuvable.', None, 2)

In [3]:
#import needed


#inputs
#from global nav
next_point=(1.00,1.00)
#from filter
actual_point=(0.00,0.00)
actual_angle=0.00

#needed variables definition
#translation: speed approx 20cm/s, need to know the coefficient linking coordinates to meters
coeff_dist=1
run_speed=100
coeff_speed=21.73913043478261
#rotation
#rot_coeff=
rot_speed=100
coeff_rotspeed=1

#motion functions
def forward(next,actual):
    dist=np.sqrt(np.sum(np.square(next-actual)))
    forward_time=(dist*coeff_dist)/(run_speed*coeff_speed)
    th.set_var("motor.left.target",run_speed)
    th.set_var("motor.right.target",run_speed)
    time.sleep(forward_time)
    stopmotors()

#can be optimised in order to choose which rotation direction is the shortest
def turn(next,actual,actual_angle):
    direction=np.subtract(next,actual)
    new_angle=np.arctan2(direction[0],direction[1])
    """"
    if((direction[0])and(not direction[1])):
        new_angle=np.arctan2(direction,(0,0))[0]
    elif((not direction[0])and(not direction[1])):
        new_angle=np.arctan2(direction,(0,0))[1]
    elif(direction[0] and direction[1]):
        new_angle=np.arctan2(direction,(0,0))[2]
    elif((not direction[0])and(direction[1])):
        new_angle=np.arctan2(direction,(0,0))[3]
    """
    
    angle_diff=new_angle-actual_angle #in radians
    rot_time=(angle_diff)/(rot_speed*coeff_rotspeed)
    if(angle_diff>0):
        th.set_var("motor.left.target",2**16-rot_speed)
        th.set_var("motor.right.target",rot_speed)
    elif(angle_diff<0):
        th.set_var("motor.left.target",rot_speed)
        th.set_var("motor.right.target",2**16-rot_speed)
    time.sleep(rot_time)
    stopmotors()

def stopmotors():
    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

In [4]:
turn(next_point,actual_point,0)

NameError: name 'th' is not defined